# Log a pytorch model

In this guide we will:
- Define a simple pytorch model
- Set a model signature for our logged model to define inputs and outputs to the mlflow model
- Log our model to MLflow server
- Load the model back from storage, to use in other notebooks

Install dependencies

In [ ]:
%pip install mlflow torch
import warnings

warnings.filterwarnings("ignore")

Import packages

In [2]:
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

import mlflow
import mlflow.pytorch

## Prepare the data

The Iris dataset is a popular beginner's dataset for classification models that contains measurements of 3 species of Iris flowers. If you want, more information can be found [at this link](https://archive.ics.uci.edu/dataset/53/iris).

 We are loading the data, standardizing it, splitting it into training and testing sets, converting it into the format required by PyTorch, and preparing it for efficient training in mini-batches.

In [3]:
# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=16)

Define your pytorch model

In [4]:
# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = SimpleNN()
criterion = nn.CrossEntropyLoss()

Define a simple accuracy metric

In [5]:
# Function to calculate accuracy
def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_true).sum().item()
    accuracy = correct / y_true.size(0)
    return accuracy

# Define the model signature
[Link to the `mlflow.models.infer_signature` docs](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.infer_signature)

A model signature defines valid input and output schema and is used to validate input and outputs at inference time. `mlflow.models.infer_signature` helps with this and creates a `ModelSignature` object that is passed into the `mlflow.pytorch.log_model`.

Since pytorch model's usually operate on tensors, we need to convert both the input and output into a type compatible with `mlflow.models.infer_signature`. Commonly, this means converting them into a `numpy.ndarray` or a dictionary of arrays (if the output is multiple tensors).

If you've already logged a model, you can add a signature to the logged model with [this API](https://www.mlflow.org/docs/2.8.0/models.html#set-signature-on-logged-model) as well.

In [6]:
from mlflow.models.signature import infer_signature

# Infer the signature of the model
sample_input = X_train_tensor[:1]
model.eval()
with torch.no_grad():
    sample_output = model(sample_input)
signature = infer_signature(sample_input.numpy(), sample_output.numpy())

print("Model signature:", signature)

Model signature: inputs: 
  [Tensor('float32', (-1, 4))]
outputs: 
  [Tensor('float32', (-1, 3))]
params: 
  None



Start an mlflow run and see how our model performs!

In [7]:
mlflow.set_experiment("iris_classification_pytorch")

# Start an MLflow run
with mlflow.start_run() as run:
    print("MLflow run ID:", run.info.run_id)
    for epoch in range(10):  # number of epochs
        total_loss = 0
        total_accuracy = 0

        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            total_loss += loss.item()
            total_accuracy += calculate_accuracy(outputs, labels)

        avg_loss = total_loss / len(train_loader)
        avg_accuracy = total_accuracy / len(train_loader)

        print(f"Epoch {epoch + 1}, Loss: {avg_loss}, Accuracy: {avg_accuracy}")
        mlflow.log_metric("loss", avg_loss, step=epoch)
        mlflow.log_metric("accuracy", avg_accuracy, step=epoch)

    # Log the PyTorch model with the signature
    mlflow.pytorch.log_model(model, "model", signature=signature)

    # Log parameters
    mlflow.log_param("epochs", 10)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("learning_rate", 0.001)

print("Model training and logging complete.")

MLflow run ID: f53ac10fd1e645c08e6a88fb79b90680
Epoch 1, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 2, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 3, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 4, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 5, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 6, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 7, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 8, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 9, Loss: 1.0173038616776466, Accuracy: 0.3828125
Epoch 10, Loss: 1.0173038616776466, Accuracy: 0.3828125
Model training and logging complete.


Loading the logged model back into memory

In [8]:
print("Run id from the run above:", run.info.run_id)

# Later, or in a different script, you can load the model using the run ID
loaded_model = mlflow.pytorch.load_model(f"runs:/{run.info.run_id}/model")

# you can now use the loaded model as you would've used the original pytorch model!
loaded_model.eval()
with torch.no_grad():
    sample_input = X_test_tensor[:1]
    loaded_output = loaded_model(sample_input)
    og_output = model(sample_input)
    print("Original model output:", og_output)
    print("Loaded model output:", loaded_output)

Run id from the run above: f53ac10fd1e645c08e6a88fb79b90680
Original model output: tensor([[ 0.1117,  0.0059, -0.0931]])
Loaded model output: tensor([[ 0.1117,  0.0059, -0.0931]])



# What you see in the mlflow UI
Please follow [this section of the contributing guide](https://github.com/mlflow/mlflow/blob/master/CONTRIBUTING.md#javascript-and-ui) to get the UI set up.
NOTE: make sure to launch `mlflow ui` in the same directory that you're running the notebook (where the `mlruns` folder is created as well)

### Experiment page
Here, you can select the experiment you set in the code above and choose a run to view the model logged during that run. You can also see how your pytorch model has changed in accuracy / loss over different runs in the `Chart` tab.

<img src="./screenshots/experiment.png" onerror="this.onerror=null; this.src='https://i.imgur.com/hiolwEe.png'" style="width: 60%">

### Runs detail page
Here, you can see the run ID of this run (used to retrieve the logged model) and the model signature that we set above.

<img src="./screenshots/run.png" onerror="this.onerror=null; this.src='https://i.imgur.com/gJN4f2v.png'" style="width: 60%">

